In [ ]:
!git clone --branch main --single-branch https://github.com/ValenSaZu/AST-Transformer.git /content/AST-Transformer

In [ ]:
!apt-get -y update -qq
!apt-get -y install -qq unar unrar p7zip-full

In [ ]:
# Descarga a /content/melaudis/raw y extrae a /content/melaudis/wav
import os, requests, glob, zipfile, subprocess

ART_ID = 27115870
ROOT = "/content/melaudis"; RAW=f"{ROOT}/raw"; WAV=f"{ROOT}/wav"
os.makedirs(RAW, exist_ok=True); os.makedirs(WAV, exist_ok=True)

# 1) listar y bajar archivos del artículo
meta = requests.get(f"https://api.figshare.com/v2/articles/{ART_ID}").json()
files = meta.get("files", [])
print(f"{len(files)} archivos en Figshare")

for f in files:
    url, name = f["download_url"], f["name"]
    out = f"{RAW}/{name}"
    if os.path.exists(out):
        print("✓ ya existe:", name); continue
    print("↓", name)
    with requests.get(url, stream=True) as r, open(out, "wb") as w:
        r.raise_for_status()
        for ch in r.iter_content(1024*1024):
            if ch: w.write(ch)

# 2) extraer .rar (usar 'unar' / 'unrar'); si hubiera .zip, también se extraen
for rar in glob.glob(f"{RAW}/*.rar"):
    print("Extract:", os.path.basename(rar))
    # intenta con unar; si falla, usa unrar
    res = subprocess.run(["unar","-force-overwrite","-o",WAV,rar])
    if res.returncode != 0:
        subprocess.run(["unrar","x","-o+","-y",rar,WAV+"/"], check=True)

for z in glob.glob(f"{RAW}/*.zip"):
    print("Unzip:", os.path.basename(z))
    with zipfile.ZipFile(z) as zf:
        zf.extractall(WAV)

# 3) conteo rápido
wavs = glob.glob(f"{WAV}/**/*.wav", recursive=True) + glob.glob(f"{WAV}/**/*.WAV", recursive=True)
print("WAVs encontrados:", len(wavs))
print("Ejemplos:", wavs[:3])

In [ ]:
%cd /content/AST-Transformer/AST-Transformer
%pip install -q -r python/requirements.txt

In [ ]:
PYTHONPATH=$PWD python scripts/make_mels.py \
  --src /content/melaudis/wav \
  --dst /content/melaudis/processed \
  --durationSec 2.0 \
  --normalize peak \
  --targetNumFrames 256 \
  --patch 16